In [1]:
# -----------------------------------------------------------
# Released under MIT License
# (C) Alexander Isaychikov, Minsk, Belarus
# email alipheesa@gmail.com
# -----------------------------------------------------------

In [2]:
import numpy as np
import math
import itertools

In [3]:
class Variable:
    """
    The Variable class instances serve as building blocks in computation graph

    Parameters
    ----------
    idf : str
        Every Variable class instance must have its identifier (see examples below)
    _prev : list
        List of previous node(s) in computation graph
    _operand_list : list
        List of all registered operands in computation graph

    Variables
    ---------
    idf : str
        Stores identifier
    _prev : list
        Stores tuple of previous nodes
    _operand_list : list
        Stores operand list

    Notes
    -----
    Note that Variable's __call__ function receives not a list, but
    a dict with every operand's identifier as key and 0/1 as value
    example: {'A':0, 'B':1, 'C':0}
    """

    def __init__(self, idf, _prev=None, _operand_list=None):

        self.idf = idf
        self._prev = _prev

        if _operand_list is None:
            if idf not in ['&', '|', '~']:
                self._operand_list = [idf]
        else:
            self._operand_list = list(set(_operand_list))

        self._operand_list.sort()

    def __call__(self, bool_dict):
        if self.idf == '&':
            return self._prev[0](bool_dict) & self._prev[1](bool_dict)
        elif self.idf == '|':
            return self._prev[0](bool_dict) | self._prev[1](bool_dict)
        elif self.idf == '~':
            return 0 if self._prev[0](bool_dict) else 1
        else:
            return bool_dict[self.idf]

    def __or__(self, other):
        return Variable('|', [self, other], (self._operand_list + other._operand_list))

    def __and__(self, other):
        return Variable('&', [self, other], (self._operand_list + other._operand_list))

    def __invert__(self):
        return Variable('~', [self], self._operand_list)

    def get_operand_list(self):
        return self._operand_list

In [4]:
def _cartesian(arrays):
    """
    Computes all existing combinations of given arrays

    Parameters
    ----------
    arrays : list
        List of arrays to combine

    Returns
    -------
    list
        List of all combinations
    """

    arrays = [np.asarray(a) for a in arrays]
    shape = (len(x) for x in arrays)

    ix = np.indices(shape, dtype=int)
    ix = ix.reshape(len(arrays), -1).T

    for n, arr in enumerate(arrays):
        ix[:, n] = arrays[n][ix[:, n]]

    return ix

def _expand_vector_to_table(vector):
    """
    Expands vector to truth table, expects vector length to be in set of power of two values
    """
    length = len(vector)
    if math.log(length, 2) != int(math.log(length, 2)):
        raise Exception('Log[2] of input vector\'s length must be integer, not float')
        
    operand_count = int(math.log(length, 2))
    lists = _cartesian([[0, 1] for _ in range(operand_count)])
    output = [list(x) + y for x, y in zip(lists, vector)]
    return output

In [5]:
def get_truth_table(function):
    """
    Computes truth table of given logical function, outputs
    list of lists with size of 2^n, where n is size of functions'
    operand list. Each list contains operand values of certain
    combination and function output, with operand's value position
    corresponding to its position in Variable's sorted operand list

    Parameters
    ----------
    function : Variable
        Logical function represented as computation graph

    Returns
    -------
    list
        Matrix (list of lists) representing truth table

    """

    output = list()
    operand_list = function.get_operand_list()
    length = len(operand_list)

    def _get_bool_dict_list():
        """
        Short function to put every combination into required dict format
        """
        lists = _cartesian([[0, 1] for _ in range(length)])
        dicts = []
        for j in range(len(lists)):
            dict_entry = {operand_list[i]: lists[j][i] for i in range(length)}
            dicts.append(dict_entry)
        return dicts

    dict_list = _get_bool_dict_list()

    for combination in dict_list:
        out = []
        for value in combination.values():
            out.append(value)
        out.append(function(combination))
        output.append(out)

    return output
        
def get_index_form(function):
    """
    Computes index form for given function, which basically
    means converting truth table's function outputs into binary
    format and then into decimal

    Parameters
    ----------
    function : Variable
        Logical function represented as computation graph

    Returns
    -------
    int
        Index form of logical function

    """
    table = get_truth_table(function)
    
    binary = [x[-1] for x in table]
    
    decimal = sum([x*pow(2, i) for i, x in enumerate(reversed(binary))])
    
    return decimal

In [6]:
def print_truth_table(function):
    """
    Simple function to print truth table from a given function
    """
    table = get_truth_table(function)

    output = list()
    operand_list = function.get_operand_list()
    length = len(operand_list)

    def _get_operand_string():
        out = ""
        for x in operand_list:
            out += x + ' '
        out += 'f('
        for i, x in enumerate(operand_list):
            if i < length - 1:
                out += x + ', '
            else:
                out += x
        out += ')'
        return out

    print(_get_operand_string())

    for combination in table:
        out = ""
        for v in combination:
            out += str(v) + ' '
        print(out)

In [7]:
def _get_sorted_input(input_list):
    """
    Split input list by amount of true values (1) in each combination

    Parameters
    ----------
    input_list : list
        Matrix (list of lists) of combinations

    Returns
    -------
    list
        list of combination Matrices
    """
    input_list = np.array(input_list)
    n = len(input_list[0])
    sum_list = (input_list == 1).sum(axis=1)
    sorted_list = []

    for i in range(n + 1):
        temp = input_list[sum_list == i]
        sorted_list.append(list(temp))
    
    sorted_list.append([])
    
    return sorted_list


def _implicant_forward(input_list, mint_to_i_dict=None):
    """
    Messy function that makes an iteration according to Quine–McCluskey algorithm

    Parameters
    ----------
    input_list : list
        list of combination Matrices, where combinations are splitted
        according to the amount of true values (1) in them
    mint_to_i_dict : dict


    Returns
    -------
    list, dict
        list of combinations
        dict of minterms

    Notes
    -----
    Forwards input list and maps calculated combinations into minterm indices,
    outputted minterm dict is used afterwards to map reduced minterm list back into
    implicant list
    """

    def _get_operand_count():
        return max([max([len(y) for y in x]) for x in input_list if ([len(y) for y in x]) != []])

    if mint_to_i_dict is None:
        mint_to_i = {tuple(v): tuple([k]) for k, v in
                     enumerate(_cartesian([[0, 1] for _ in range(_get_operand_count())]))}
    else:
        mint_to_i = mint_to_i_dict

    output = []
    output_minterm = mint_to_i if mint_to_i_dict is None else mint_to_i_dict

    for j in range(len(input_list) - 1):
        for x in input_list[j]:

            is_combinable = False

            for y in input_list[j + 1]:
                def _handle_combination():
                    counter = 0
                    idx = 0
                    comb = list(y)
                    for i in range(len(x)):
                        if y[i] not in [x[i]]:
                            counter += 1
                            idx = i
                    if counter == 1:
                        comb[idx] = -1
                        if comb not in output:
                            output.append(comb)
                            output_minterm[tuple(comb)] = tuple(set([k for k in mint_to_i[tuple(x)]] +
                                                                    [k for k in mint_to_i[tuple(y)]]))
                        is_combinable = True

                _handle_combination()

            if not is_combinable:
                output.append(list(x))

    return output, output_minterm


def _remove_implicant_duplicates(input_list):
    """
    Recieves a list of implicants and removes duplicate implicants
    of smaller size

    Parameters
    ----------
    input_list : list
        list of implicants

    Returns
    -------
    list
        reduced list of implicants
    """
    output = []
    for x in input_list:
        is_unique = True
        for y in input_list:
            if x == y:
                continue
            if is_unique == False:
                continue
            counter_01 = 0
            counter_01m1 = 0
            counter_m101 = 0
            for i in range(len(x)):
                if x[i] == y[i]:
                    continue
                elif x[i] != -1 and y[i] != -1:
                    counter_01 += 1
                elif x[i] == -1:
                    counter_01m1 += 1
                elif y[i] == -1:
                    counter_m101 += 1

            if counter_01 == 0 and counter_01m1 == 0 and counter_m101 > 0:
                is_unique = False

        if is_unique == True:
            output.append(x)

    return output


def _remove_minterm_duplicates(input_list, minterm_dict):
    """
    Truncates minterm dict by removing keys that are not in implicant list
    """
    return {k: v for k, v in minterm_dict.items() if list(k) in input_list}


def _get_implicant_list(input_list):
    """
    Iterates according to Quine–McCluskey algorithm multiple times,
    outputs implicant list and minterm dict

    Combines all declared above functions
    """
    minterm_dict = None
    for i in range(len(input_list[0])):
        input_list, minterm_dict = _implicant_forward(_get_sorted_input(input_list), minterm_dict)

    output_list = _remove_implicant_duplicates(input_list)
    output_minterm = _remove_minterm_duplicates(output_list, minterm_dict)

    return output_list, output_minterm


def _get_reduced_implicant_list(input_list):
    """
    Reduces implicant list to contain all minterms,
    outputs the smallest minterm combination
    """
    input_list, minterm_dict = _get_implicant_list(input_list)
    minterm_set = set([y for x in minterm_dict.values() for y in x])
    key_combinations = []
    value_combinations = []

    for length in range(len(input_list[0]) + 1):
        for combination in itertools.combinations(list(minterm_dict.keys()), length):
            key_combinations.append(combination)
        for combination in itertools.combinations(list(minterm_dict.values()), length):
            value_combinations.append(set([y for x in combination for y in x]))

    output = [x for i, x in enumerate(key_combinations) if value_combinations[i] == minterm_set]
    if len(output) == 0:
        return []
    output = [x for x in output if len(x) == min([len(y) for y in output])]
    output = [list(x) for x in output[0]]

    return output

In [8]:
def build_PDNF(value, custom_operands=None):
    """
    Builds Principal Disjunction Normal Form of a given function or truth table

    Parameters
    ----------
    value : Variable/list
        Input function or truth table's single output vector
    custom_operands: list
        Custom operand identifiers (instead of standard A, B, C etc.)
        
    Returns
    -------
    str
        String representation of PDNF
    """
    if isinstance(value, Variable):
        table = get_truth_table(value)
    else:
        table = _expand_vector_to_table(value)
    
    if custom_operands is not None:
        operands = custom_operands
    elif isinstance(value, Variable):
        operands = value._operand_list
    else:
        operands = [chr(65 + i) for i in range(len(table[0]) - 1)]
        
    combinations = [c[:-1] for c in table if c[-1] == 1]
    if len(combinations) == 0:
        return 'PDNF does not exist'
    out = ""

    for i, c in enumerate(combinations):
        def get_minterm_str(combination):
            temp = ""
            for i, operand in enumerate(operands):
                temp += '' if combination[i] == 1 else '~'
                temp += operand
                temp += '&' if i < len(operands) - 1 else ''
            return temp

        out += '(' + get_minterm_str(c) + ')'
        out += ' | ' if i < len(combinations) - 1 else ''

    return out


def build_PCNF(value, custom_operands=None):
    """
    Builds Conjunctive Disjunction Normal Form of a given function or truth table

    Parameters
    ----------
    value : Variable/list
        Input function or truth table's single output vector
    custom_operands: list
        Custom operand identifiers (instead of standard A, B, C etc.)
        
    Returns
    -------
    str
        String representation of PCNF
    """
    if isinstance(value, Variable):
        table = get_truth_table(value)
    else:
        table = _expand_vector_to_table(value)
    
    if custom_operands is not None:
        operands = custom_operands
    elif isinstance(value, Variable):
        operands = value._operand_list
    else:
        operands = [chr(65 + i) for i in range(len(table[0]) - 1)]
        
        
    combinations = [c[:-1] for c in table if c[-1] == 0]
    if len(combinations) == 0:
        return 'PCNF does not exist'
    out = ""

    for i, c in enumerate(combinations):
        def get_minterm_str(combination):
            temp = ""
            for i, operand in enumerate(operands):
                temp += '' if combination[i] == 0 else '~'
                temp += operand
                temp += '|' if i < len(operands) - 1 else ''
            return temp

        out += '(' + get_minterm_str(c) + ')'
        out += ' & ' if i < len(combinations) - 1 else ''

    return out


def minimize_PDNF(value, custom_operands=None):
    """
    Builds minimization for PDNF of a given function or truth table

    Parameters
    ----------
    value : Variable/list
        Input function or truth table's single output vector
    custom_operands: list
        Custom operand identifiers (instead of standard A, B, C etc.)
        
    Returns
    -------
    str
        String representation of PDNF minimization
    """
    if isinstance(value, Variable):
        table = get_truth_table(value)
    else:
        table = _expand_vector_to_table(value)
    
    if custom_operands is not None:
        operands = custom_operands
    elif isinstance(value, Variable):
        operands = value._operand_list
    else:
        operands = [chr(65 + i) for i in range(len(table[0]) - 1)]
        
    combinations = [c[:-1] for c in table if c[-1] == 1]
    if len(combinations) == 0:
        return 'PDNF minimization does not exist'
    combinations = _get_reduced_implicant_list(combinations)
    if len(combinations) == 0:
        return build_PDNF(value, custom_operands)
    out = ""

    for current_index, combination in enumerate(combinations):
        def get_minterm_str(combination):
            string = ""
            operator_count = len([x for x in combination if x != -1]) - 1
            for i, operand in enumerate(operands):
                string += '~' if combination[i] == 0 else ''
                string += operand if combination[i] != -1 else ''
                string += '&' if (operator_count > 0 and combination[i] != -1) else ''
                if operator_count > 0 and combination[i] != -1:
                    operator_count -= 1

            return string

        out += '(' + get_minterm_str(combination) + ')'
        out += ' | ' if current_index < len(combinations) - 1 else ''

    return out


def minimize_PCNF(value, custom_operands=None):
    """
    Builds minimization for PCNF of a given function or truth table

    Parameters
    ----------
    value : Variable/list
        Input function or truth table's single output vector
    custom_operands: list
        Custom operand identifiers (instead of standard A, B, C etc.)
        
    Returns
    -------
    str
        String representation of PCNF minimization
    """
    if isinstance(value, Variable):
        table = get_truth_table(value)
    else:
        table = _expand_vector_to_table(value)
    
    if custom_operands is not None:
        operands = custom_operands
    elif isinstance(value, Variable):
        operands = value._operand_list
    else:
        operands = [chr(65 + i) for i in range(len(table[0]) - 1)]

    combinations = [c[:-1] for c in table if c[-1] == 0]
    if len(combinations) == 0:
        return 'PCNF minimization does not exist'
    combinations = _get_reduced_implicant_list(combinations)
    if len(combinations) == 0:
        return build_PCNF(value, custom_operands)
    out = ""

    for n, c in enumerate(combinations):
        def get_minterm_str(combination):
            string = ""
            operator_count = len([x for x in combination if x != -1]) - 1
            for i, operand in enumerate(operands):
                string += '~' if combination[i] == 1 else ''
                string += operand if combination[i] != -1 else ''
                string += '|' if (operator_count > 0 and combination[i] != -1) else ''
                if operator_count > 0 and combination[i] != -1:
                    operator_count -= 1

            return string

        out += '(' + get_minterm_str(c) + ')'
        out += ' & ' if n < len(combinations) - 1 else ''

    return out

#  Examples

In [11]:
A = Variable('A')
B = Variable('B')
C = Variable('C')
D = Variable('D')
E = Variable('E')

# out = ( (C | B & E) & (A | D) & E) & E
out = ~( (A | B) & (A & ~C) )

In [12]:
print('-'*100)
print_truth_table(out)
print('-'*100)
print('Index form: ', get_index_form(out))
print('-'*100)
print('PDNF', build_PDNF(out))
print('-'*100)
print('PCNF', build_PCNF(out))
print('-'*100)
print('PDNF minimization', minimize_PDNF(out))
print('-'*100)
print('PCNF minimization', minimize_PCNF(out))
print('-'*100)

----------------------------------------------------------------------------------------------------
A B C f(A, B, C)
0 0 0 1 
0 0 1 1 
0 1 0 1 
0 1 1 1 
1 0 0 0 
1 0 1 1 
1 1 0 0 
1 1 1 1 
----------------------------------------------------------------------------------------------------
Index form:  245
----------------------------------------------------------------------------------------------------
PDNF (~A&~B&~C) | (~A&~B&C) | (~A&B&~C) | (~A&B&C) | (A&~B&C) | (A&B&C)
----------------------------------------------------------------------------------------------------
PCNF (~A|B|C) & (~A|~B|C)
----------------------------------------------------------------------------------------------------
PDNF minimization (~A) | (C)
----------------------------------------------------------------------------------------------------
PCNF minimization (~A|C)
----------------------------------------------------------------------------------------------------
